In [76]:
from pymongo import MongoClient

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {
            "title": "Kafka on the Shore",
            "author": "Haruki Murakami",
            "year": 2002,
            "genre": "fantasy",
        },
        {
            "title": "Norwegian Wood",
            "author": "Haruki Murakami",
            "year": 1987,
            "genre": "thriller",
        },
        {
            "title": "1Q84",
            "author": "Haruki Murakami",
            "year": 2009,
            "genre": "fantasy",
        },
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {
            "title": "Inception",
            "director": "Christopher Nolan",
            "year": 2010,
            "rating": 8.8,
            "genre": "fantasy",
        },
        {
            "title": "Interstellar",
            "director": "Christopher Nolan",
            "year": 2014,
            "rating": 8.6,
            "genre": "fantasy",
        },
        {
            "title": "The Dark Knight",
            "director": "Christopher Nolan",
            "year": 2008,
            "rating": 9.0,
            "genre": "fantasy",
        },
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": "2023-04-12T08:00:00Z"},
        {"user_id": 1, "action": "view", "timestamp": "2023-04-09T09:00:00Z"},
        {"user_id": 2, "action": "purchase", "timestamp": "2023-04-12T10:00:00Z"}
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


Data inserted successfully


In [77]:
# [문제 1: 특정 장르의 책 찾기]

def find_books_by_genre(db, genre):
    books_collection = db.books
    query = {"genre": genre}
    projection = {"_id": 0, "title": 1, "author": 1}

    books = books_collection.find(query, projection)
    for book in books:
        print(book)

find_books_by_genre(db, "fantasy")


{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}


In [78]:
# [문제 2: 감독별 평균 영화 평점 계산]

def calculate_average_ratings(db):
    movie_collection = db.movies
    query = [{"$group": {"_id": "$director", "avgRating": {"$avg": "$rating"}}},{"$sort":{"director":-1}}]

    ratings = movie_collection.aggregate(query)
    for rating in ratings:
        print(rating)
        
calculate_average_ratings(db)

{'_id': 'Christopher Nolan', 'avgRating': 8.799999999999999}


In [79]:
# [문제 3: 특정 사용자의 최근 행동 조회]

def find_user_action(db,user_id,limit):
    action_collection = db.user_actions
    query = {"user_id":user_id}
    projection = [("timestamp",-1)]
    
    
    actions = action_collection.find(query).sort(projection).limit(limit)
    for action in actions:
        print(action)
        
find_user_action(db, 1, 5)

{'_id': ObjectId('6630a11a7d82c2bed71438d0'), 'user_id': 1, 'action': 'click', 'timestamp': '2023-04-12T08:00:00Z'}
{'_id': ObjectId('6630a11a7d82c2bed71438d1'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-09T09:00:00Z'}


In [80]:
# [문제 4: 출판 연도별 책의 수 계산]

def count_num_of_books(db):
    books_collection = db.books
    query = [{"$group":{"_id":"$year","count":{"$sum":1}}},{"$sort":{"count":-1}}]
    
    books = books_collection.aggregate(query)
    for book in books:
        print(book)
        
count_num_of_books(db)

{'_id': 1987, 'count': 1}
{'_id': 2002, 'count': 1}
{'_id': 2009, 'count': 1}


In [81]:
# [문제 5: 특정 사용자의 행동 유형 업데이트]

from datetime import datetime

def update_user_actions_before_date(db, user_id, date, old_action, new_action):
    actions_collection = db.user_actions
    query = {"user_id": user_id, "action": old_action, "timestamp": {"$lt": date}}
    update = {"$set": {"action": new_action}}

    result = actions_collection.update_many(query, update)
    print(f"Updated {result.modified_count} documents.")

update_user_actions_before_date(db, 1, datetime(2023, 4, 10), "view", "seen")

Updated 0 documents.
